## HOMEWORK MODULE 1

In [1]:
import pandas as pd
import numpy as np

In [2]:
dfJan = pd.read_parquet('fhv_tripdata_2021-01.parquet')
dfFeb = pd.read_parquet('fhv_tripdata_2021-02.parquet')

In [4]:
dfJan.shape

(1154112, 7)

In [5]:
dfJan['duration'] = dfJan.dropOff_datetime - dfJan.pickup_datetime
dfJan.duration = dfJan.duration.apply(lambda td: td.total_seconds() / 60)

In [6]:
dfJan.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,110.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667


In [7]:
dfJan["duration"].mean()

19.1672240937939

In [8]:
dfJan = dfJan[(dfJan.duration >= 1) & (dfJan.duration <= 60)]

In [10]:
dfJan["PUlocationID"].isnull().sum() * 100 / len(dfJan)

83.52732770722618

In [16]:
dfJan["PUlocationID"] = dfJan["PUlocationID"].fillna(-1)
dfJan["DOlocationID"] = dfJan["DOlocationID"].fillna(-1)

In [17]:
dfJan.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,9.050000


In [22]:
categorical = ['PUlocationID', 'DOlocationID']
# set categorical items to string for encoding
dfJan[categorical] = dfJan[categorical].astype(str)

train_dicts = dfJan[categorical].to_dict(orient='records')

In [ ]:
from sklearn.feature_extraction import DictVectorizer

# apply encoder to dictionary
dv = DictVectorizer()
# set the encoded dictionary to train data
X_train = dv.fit_transform(train_dicts)

print(X_train)

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# define the target variable
target = "duration"
# set the target value to y_train
y_train = dfJan[target].values

#fit a linearregression model on the training data
lr = LinearRegression()
lr.fit(X_train, y_train)

# set prediction to training data
y_pred = lr.predict(X_train)

#compute the mean squared error
mean_squared_error(y_train, y_pred, squared=False)

10.528519107210744

In [56]:
# function to do the above
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.dropoff_datetime = pd.to_datetime(df.dropoff_datetime)
        df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df["PUlocationID"] = df["PUlocationID"].fillna(-1)
    df["DOlocationID"] = df["DOlocationID"].fillna(-1)

    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [57]:
# FEB AS VAL DATA
df_train = read_dataframe('fhv_tripdata_2021-01.parquet')
df_val = read_dataframe('fhv_tripdata_2021-02.parquet')

In [60]:
df_train['PU_DO'] = df_train['PUlocationID'] + '_' + df_train['DOlocationID']
df_val['PU_DO'] = df_val['PUlocationID'] + '_' + df_val['DOlocationID']

In [61]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']

train_dicts2 = df_train[categorical].to_dict(orient='records')
X_train2 = dv.fit_transform(train_dicts2)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [62]:
target = "duration"
y_train2 = df_train[target].values
y_val = df_val[target].values

In [63]:
lr = LinearRegression()
lr.fit(X_train2, y_train2)

y_pred2 = lr.predict(X_val)

mean_squared_error(y_val, y_pred2, squared=False)

10.596293672454877